<a href="https://www.kaggle.com/fanbyprinciple/automated-scoring-of-autobiographical-interviews?scriptVersionId=88122779" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Welcome!

from : https://colab.research.google.com/github/rubenvangenugten/autobiographical_interview_scoring/blob/main/automated_autobiographical_interview_scoring_share.ipynb#scrollTo=8WKHko67Ht6s

The Autobiographical Interview has been used in more than two hundred studies to assess the content of autobiographical memories. In a typical experiment, participants recall memories, which are then scored manually for internal details (episodic details from the central event) and external details (largely non-episodic details). Scoring these narratives requires a significant amount of time.

![](https://study.com/cimages/videopreview/videopreview-full/literary-screenshot_133867.jpg)

Please read through the .readme file on github to ensure your data is in the correct format. 

Now, to run the code, click 'runtime' -> 'run all' in your menu bar. You'll be prompted for google drive login, which colab needs to access your data file. Once the code is done running, you should be able to find the results in the folder you created. 


All code for model loading, use, data encoding, and related matters has been copied or adapted from the huggingface [documentation](https://huggingface.co/docs/transformers/index) and the example [library](https://github.com/huggingface/notebooks/blob/master/transformers_doc/training.ipynb)


Edit the cell below to provide the data file location and the location where you want the results stored.


In [1]:
!wget "https://raw.githubusercontent.com/rubenvangenugten/autobiographical_interview_scoring/main/example_data.csv"

--2022-02-18 01:49:14--  https://raw.githubusercontent.com/rubenvangenugten/autobiographical_interview_scoring/main/example_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 673 [text/plain]
Saving to: ‘example_data.csv’

example_data.csv    100%[===================>]     673  --.-KB/s    in 0s      

2022-02-18 01:49:15 (34.8 MB/s) - ‘example_data.csv’ saved [673/673]



In [2]:
data_location = './example_data.csv'
output_location = './'

The code below installs necessary libraries

In [3]:
pip install transformers==4.6.0

     |████████████████████████████████| 2.3 MB 889 kB/s            
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.1.2
    Uninstalling huggingface-hub-0.1.2:
      Successfully uninstalled huggingface-hub-0.1.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.12.5
    Uninstalling transformers-4.12.5:
      Successfully uninstalled transformers-4.12.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.18.0 requires huggingface-hub<1.0.0,>=0.1.0, but you have huggingface-hub 0.0.8 which is incompatible.
cached-path 0.3.2 requires huggingface-hub<0.2.0,>=0.0.12, but you have huggingface-hub 0.0.8 which is incompatible.
allennlp 2.8.0 requires huggingface-hub>=0.0.16, but you have huggingface-hub 0.0.8 which is incompatible.
Note: you may need to restart the kernel to use upda

In [4]:
pip install pysbd

     |████████████████████████████████| 71 kB 891 kB/s            
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip -q install datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.6.0 requires huggingface-hub==0.0.8, but you have huggingface-hub 0.4.0 which is incompatible.
cached-path 0.3.2 requires huggingface-hub<0.2.0,>=0.0.12, but you have huggingface-hub 0.4.0 which is incompatible.


In [6]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
import pysbd
import re
from transformers import AutoTokenizer, TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

define sentence segmenter so that it's easier to use later


In [7]:
seg = pysbd.Segmenter(language="en", clean=False)

Reload model trained on all datasets. 
Reload tokenizer saved with it (just standard distilbert tokenizer). These are stored on the huggingface model hub and can be accessed directly with code below

In [8]:
model = TFDistilBertForSequenceClassification.from_pretrained("vangenugtenr/autobiographical_interview_scoring")
tokenizer = AutoTokenizer.from_pretrained("vangenugtenr/autobiographical_interview_scoring")

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]


User settings:

   KMP_AFFINITY=granularity=fine,noverbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Read in data from the location provided at the top of this notebook

In [9]:
allDat = pd.read_csv(data_location)

In [10]:
allDat.iloc[0].text

'We tend to go to the beach every year for my birthday. I don’t really remember the location. But I was probably ten years old, because I think we were with my childhood friend john. There are always people laying on the sand, so I assume there are in this memory too. This memory makes me want to go to the beach now.'

In [11]:
allDat.head()

,participantID,prompt,text
0,1,1,We tend to go to the beach every year for my b...
1,2,1,I remember laying on the sand. I had a cold be...


In [12]:
allDat.columns

Index(['participantID', 'prompt', 'text'], dtype='object')

Sometimes CSV files that you read in have rows with NAs at the end. let's remove those empty rows (and any rows with missing data in participantID, prompt, and text)

In [13]:
allDat.dropna(subset = ["participantID", "prompt", 'text'], inplace=True)
#allDat.dropna(subset = ["participantID", 'text'], inplace=True)

Now, take each narrative and split it up into sentences. We do this because sentences are the level at which we classify content. We want to end up with a new dataframe in which each row contains participantID, prompt, text, and one  sentence. So let's say we start with two initial rows of data: two narratives. Let's say that each has ten sentences. After the code below, our resulting dataframe should be 20 rows long.

This way, the sentences can easily be classified and are matched to the stories they came from.

Approach:

Loop through each narrative, create a new dataframe where each row contains the participantID, prompt, text, and one of the resulting sentences. Store this dataframe. At the end, bind all dataframes together. This way, we obtain a dataframe with all narratives in long format, where each row has a sentence that can be classified.

In [14]:
list_of_dataframes = []

for row in range(allDat.shape[0]):
    
    # access some general info about this narrative
    this_subID = allDat.iloc[row, allDat.columns.get_loc("participantID")]
    this_prompt = allDat.iloc[row, allDat.columns.get_loc("prompt")]
    narrative = allDat.iloc[row, allDat.columns.get_loc("text")]

    # store current row
    currentRow = allDat.iloc[[row], :] # if don't have brackets around row, not returned as Df, which is needed for merge

    # create new dataframe with each row a new sentence, and subID and prompt added
    segmented_sentences = seg.segment(narrative)
    sentences_df = pd.DataFrame(segmented_sentences, columns=['sentence'])
    sentences_df["participantID"] = this_subID
    sentences_df["prompt"] = this_prompt

    # create a new merged dataframe 
    merged_thisNarrative = pd.merge(currentRow, sentences_df, on=["participantID", "prompt"])

    list_of_dataframes.append(merged_thisNarrative)
  

testData = pd.concat(list_of_dataframes)


Now, prepare the data so that it can passed to our model

In [15]:
# now, make sure data are character

testData.loc[:,'sentence'] = testData.loc[:,'sentence'].astype('str')
        
# create list of texts to classify (put in list format to encode texts)
    
test_texts = []

for row2 in range(testData.shape[0]):
    temp_test = testData.iloc[row2, testData.columns.get_loc("sentence")]
    temp_test = str(temp_test) # strip name of dataframe, then turn into string
    test_texts.append(temp_test)


# encode text into something that bert can work with.

test_encodings = tokenizer(test_texts, truncation=True, padding=True)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings)
))

Classify each sentence

In [16]:
# set up text classification pipeline using our model and tokenizer

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

# to keep amount of ram low, so that we can use the free version of
# google colab, split classification up into batches rather than all
# sentences at once.

stored_test = []
batch_size = 200  

for i in range(0, len(test_texts), batch_size):
   stored_test.extend(pipe(test_texts[i:i+batch_size]))

After the step above, predicted labels are stored in a list format. We'd like to work with a dataframe of predictions. So just loop through each list and turn it into a dataframe, then put them all together

In [17]:
list_of_predictionDfs = []

for row in range(len(stored_test)):
    
    thisTestLabels = pd.DataFrame(stored_test[row]) 
    thisTestLabels.index = thisTestLabels['label']
    thisTestLabels = thisTestLabels.drop('label', axis = 1)
    thisTestLabels = thisTestLabels.transpose()

    list_of_predictionDfs.append(thisTestLabels)

predictionsDf = pd.concat(list_of_predictionDfs)

Identify which label was most likely for the text

In [18]:
predictionsDf['toplabel'] = predictionsDf.idxmax(axis=1)

Merge predictions dataframe with the long dataframe to obtain one large dataframe

In [19]:
testData2 = pd.concat([testData.reset_index(drop=True), predictionsDf.reset_index(drop=True)], axis=1)

Add a wordcount for each sentence. use for loop for readability (plus, takes no time)

In [20]:
testData2[['sentenceWordCount']] = 0

for row in range(testData2.shape[0]):
  line = testData2.iloc[row, testData2.columns.get_loc("sentence")]
  count = len(re.findall(r'\w+', line))
  testData2.iloc[row, testData2.columns.get_loc("sentenceWordCount")] = count

Create two new columns: number of words classified as internal, and number of words classified as external. Fill in those columns. As a reminder from the paper, each sentence is classified as containing 0% internal data (i.e. 100% external data), 50% internal, 75% internal, and 100% internal data.

In [21]:
testData2[['numInt_preds']] = 0
testData2[['numExt_preds']] = 0

# now loop through each row and add in the counts

for row in range(testData2.shape[0]):
    
    predictionType_thisIter = testData2.iloc[row, testData2.columns.get_loc("toplabel")]
    numTotalWords = testData2.iloc[row, testData2.columns.get_loc("sentenceWordCount")]

    internalLocation = testData2.columns.get_loc("numInt_preds")
    externalLocation = testData2.columns.get_loc("numExt_preds")
    
    if predictionType_thisIter == 'LABEL_0':
        testData2.iloc[row, externalLocation] = numTotalWords

    if predictionType_thisIter == 'LABEL_1':
        halfDetails = numTotalWords/2
        testData2.iloc[row, externalLocation] = halfDetails
        testData2.iloc[row, internalLocation] = halfDetails

    if predictionType_thisIter == 'LABEL_2':
        testData2.iloc[row, externalLocation] = numTotalWords/4
        testData2.iloc[row, internalLocation] = numTotalWords*(3/4)
            
    if predictionType_thisIter == 'LABEL_3':
        testData2.iloc[row, internalLocation] = numTotalWords

We now have the predicted number of internal and exteranl words for each sentence. We are interested in a narrative-level summary. So, have to sum up all internal words per narrative and all external words per narrative. We do that below

In [22]:
test_write_out_subset = testData2.loc[:,["participantID","prompt", "text","numInt_preds", "numExt_preds", 'sentenceWordCount']]
grouped = test_write_out_subset.groupby(by = ["participantID", "prompt"]).agg({'text': 'first', 
                                            'numInt_preds': 'sum', 
                                            'numExt_preds': 'sum',
                                            'sentenceWordCount': 'sum'})

Since we have summed across sentences, the sentenceWordCount column actually contains the wordcount of the whole narrative. so, just rename sentenceWordCount to totalWordCount, which is more appropriate

In [23]:
grouped.rename(columns = {"sentenceWordCount": "totalWordCount"}, 
          inplace = True)

Finally, write out data to location specified at the top

In [24]:
grouped.to_csv(output_location + '/automated_autobio_scored.csv')

### lets look at the auto scored csv

In [25]:
df = pd.read_csv('./automated_autobio_scored.csv')
df.head()

,participantID,prompt,text,numInt_preds,numExt_preds,totalWordCount
0,1,1,We tend to go to the beach every year for my b...,8.5,55.5,64
1,2,1,I remember laying on the sand. I had a cold be...,52.0,15.0,67
